# E-SEIC
Selection of evolutionary instances with constraints for unbalanced datasets

## 数据集的预处理 

In [3]:
from utils.dataset_utils import get_distribution, k_fold_cross_validation
from instance_selection.parameter.parameter import *  # 导入参数的设定
from instance_selection.operator.init_toolbox import init_toolbox_eseic
from instance_selection.operator.metrics import calculate_gmean_mauc, calculate_average_accuracy, \
    calculate_average_gmean_mauc, calculate_accuracy
from instance_selection.operator.genetic_operator import selTournamentNDCD
from instance_selection.operator.ensemble import vote_result_ensembles, ensemble_individuals
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.base import clone
import scipy.io as sio  # 从.mat文件中读取数据集
import random
from deap import tools
import warnings
import numpy as np

warnings.filterwarnings("ignore")  # 忽略警告

DATASET = Satellite  # 数据集名称（包含对应参数的字典形式）
datasetname = DATASET['DATASETNAME'].split('.')[0]
mat_data = sio.loadmat('../../data/dataset/' + DATASET['DATASETNAME'])  # 加载、划分数据集
x = mat_data['X']
y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=RANDOM_SEED)  # 划分数据集
scaler = StandardScaler()  # 数据的标准化
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

unique_elements_all, classes_all, counts_all = get_distribution(y)  # 获取原始数据集分布
unique_elements_train, classes_train, counts_train = get_distribution(y_train)  # 获取训练集分布
unique_elements_test, classes_test, counts_test = get_distribution(y_test)  # 获取测试集分布
print(datasetname + f' distribution: {counts_all}')
print(f'trainset distribution: {counts_train}')
print(f'testset distribution: {counts_test}')

model = MLPClassifier(hidden_layer_sizes=(DATASET['HIDDEN_SIZE'],), max_iter=DATASET['MAX_ITER'],
                      random_state=RANDOM_SEED, learning_rate_init=DATASET['LEARNING_RATE'])

weights_train = (1 / counts_train.astype(float)) / np.sum(1 / counts_train.astype(float))  # 计算每个类的权重，用于计算每个类别的权重
weights_test = (1 / counts_test.astype(float)) / np.sum(1 / counts_test.astype(float))  # 计算每个类的权重，用于计算每个类别的权重

num_instances = int(np.ceil(counts_train.min() * 0.9))  # 取最小数量的类的0.9（向下取整）
print("最小数量:", num_instances)

y_train_pred_proba = k_fold_cross_validation(model=clone(model), X=x_train, y=y_train, n_splits=N_SPLITS, method='soft',
                                             random_state=RANDOM_SEED)  # 交叉验证得到软标签
# 将概率转化为预测结果
y_train_pred = np.argmax(y_train_pred_proba, axis=1)

Acc1, Acc2, Acc3 = calculate_accuracy(y_train_pred, y_train, weights_train)
constraints = [Acc1, Acc2, Acc3]

model.fit(x_train, y_train)
y_test_pred_proba= model.predict_proba(x_test)
print(calculate_gmean_mauc(y_test_pred_proba, y_test))

Satellite distribution: [1533  703 1358  626  707 1508]
trainset distribution: [1237  566 1078  496  568 1203]
testset distribution: [296 137 280 130 139 305]
最小数量: 447
(0.845994, 0.978635, array([0.96283784, 0.97080292, 0.82142857, 0.62307692, 0.89208633,
       0.85901639]))


## E-SEIC

In [2]:
toolbox = init_toolbox_eseic(model, x_train, y_train, weights_train, constraints, n_splits=N_SPLITS,
                             random_seed=RANDOM_SEED)  # 初始化toolbox


def main(x_train, y_train, model, balanced_method='balanced'):
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    logbook = tools.Logbook()
    logbook.header = "gen", "feasible", "ensembles_size", "avg_gmean", "avg_mauc", "avg_acc2"
    ####################################种群的初始化###########################
    pop = toolbox.population(n=POPSIZE)  # 个体编码默认全为0
    pop = toolbox.init_population(pop, balanced_method=balanced_method)  # 初始化种群中的个体
    toolbox.evaluate(pop)  # 计算个体的适应度
    ####################################种群的迭代#################################################
    for gen in range(1, NGEN + 1):
        offspring = selTournamentNDCD(pop, POPSIZE, tournsize=3)  # 锦标赛选择（1、先根据非支配排序的等级2、再根据拥挤距离）
        offspring = [toolbox.clone(ind) for ind in offspring]
        for i in range(0, len(offspring) - 1, 2):
            if random.random() <= CXPB:
                offspring[i], offspring[i + 1] = toolbox.mate(offspring[i], offspring[i + 1])  # 单点交叉
            offspring[i] = toolbox.mutate(offspring[i], MR)[0]  # 二进制反转突变
            offspring[i + 1] = toolbox.mutate(offspring[i + 1], MR)[0]  # 二进制反转突变
            del offspring[i].fitness.values, offspring[i + 1].fitness.values
        #############################################################合并、去重#####################################################
        pop = pop + offspring  # 种群的合并
        pop, _ = toolbox.remove_duplicates(pop)  # 去重
        while len(pop) < POPSIZE:  # 保证种群大小为POPSIZE
            add_individual = []
            num_add = POPSIZE - len(pop)
            for i in range(0, num_add):
                index = random.randint(0, len(offspring) - 1)  # 在0-len(offspring)范围内随机产生一个索引
                offspring[index] = toolbox.mutate(offspring[index], MR)[0]  # 选择index对应的个体进行突变
                del offspring[index].fitness.values
                add_individual.append(offspring[index])
            pop = pop + add_individual  # 种群的合并
            pop, _ = toolbox.remove_duplicates(pop)  # 去重
        toolbox.evaluate(pop)  # 计算新种群适应度
        ###############################################得到pareto_fronts############################################
        feasible_pop, infeasible_pop = toolbox.get_feasible_infeasible(pop)  # 得到可行解与不可行解
        if len(feasible_pop) >= POPSIZE:
            pop, pareto_fronts = toolbox.select(feasible_pop, POPSIZE)
            # ensembles = pareto_fronts[0]  # pareto_first_front
            ensembles = pop  # pop均为可行解，则集成pop中所有个体
        elif len(feasible_pop) > 0:
            pop = feasible_pop + infeasible_pop[:POPSIZE - len(feasible_pop)]  # 在不可行解中选取违约程度小的个体，保证pop数量为POPSIZE
            # ensembles = tools.sortNondominated(feasible_pop, len(feasible_pop))[0]  # pareto_first_front
            ensembles = feasible_pop  # 只集成可行解
        else:
            pop = feasible_pop + infeasible_pop[:POPSIZE - len(feasible_pop)]  # 加入不可行解中违约程度小的个体，保证pop数量为POPSIZE
            ensembles = [infeasible_pop[0]]  # 没有可行解，集成不可行解中第一个个体
        avg_gmean, avg_mauc = calculate_average_gmean_mauc(ensembles)  # 计算gmean、mauc的平均值
        _, avg_acc2, _ = calculate_average_accuracy(ensembles)  # 计算acc1、acc2、acc3的平均值

        record = stats.compile(pop)
        logbook.record(gen=gen, feasible=len(feasible_pop), ensembles_size=len(ensembles), avg_gmean=avg_gmean,
                       avg_mauc=avg_mauc, avg_acc2=avg_acc2, **record)
        print(logbook.stream)
    ensemble_classifiers = ensemble_individuals(ensembles, clone(model), x_train, y_train)
    return ensemble_classifiers

In [3]:
if __name__ == "__main__":
    ensemble_classifiers = main(x_train, y_train, model=model, balanced_method='random')
    print("##############################集成分类器的预测结果：################################")
    vote_pred_prob = vote_result_ensembles(ensemble_classifiers, x_test)  # 默认预测结果是软标签
    vote_pred = np.argmax(vote_pred_prob, axis=1)
    gmean, mauc, recall_per_class = calculate_gmean_mauc(vote_pred_prob, y_test)
    acc1, acc2, acc3 = calculate_accuracy(vote_pred, y_test, weights_test)
    print(f"集成分类结果：Recall{recall_per_class}，Gmean：{gmean}，mAUC：{mauc}，Acc1：{acc1}，Acc2：{acc2}，Acc3：{acc3}")
    print("训练已结束！")

gen	feasible	ensembles_size	avg_gmean	avg_mauc	avg_acc2
1  	3       	2             	0.524399 	0.706708	0.531072
2  	5       	2             	0.52626  	0.712846	0.532911
3  	7       	2             	0.52626  	0.712846	0.532911
4  	12      	3             	0.533506 	0.715837	0.538401
5  	15      	5             	0.528194 	0.712037	0.535094
6  	20      	6             	0.529644 	0.712217	0.535992
7  	25      	6             	0.529644 	0.712217	0.535992
8  	30      	6             	0.529644 	0.712217	0.535992
9  	35      	1             	0.567861 	0.72324 	0.569111
10 	40      	1             	0.567861 	0.72324 	0.569111
11 	51      	1             	0.567861 	0.72324 	0.569111
12 	48      	1             	0.567861 	0.72324 	0.569111
13 	47      	1             	0.567861 	0.72324 	0.569111
14 	53      	1             	0.567861 	0.72324 	0.569111
15 	49      	1             	0.567861 	0.72324 	0.569111
16 	51      	1             	0.581559 	0.741552	0.583183
17 	53      	1             	0.581559 	0.741552	0